In [2]:
# from transformers import AutoTokenizer

# checkpoint = "google-t5/t5-small"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)



import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device="cuda" if torch.cuda.is_available() else "cpu"

model_name="distilbert/distilgpt2"
# model_name= "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoded_input = tokenizer("Hugging Face is an open-source company", return_tensors="pt").to(device)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)




In [5]:
num_generated=6
batch_size=2
length_penalty=0
top_p=1
top_k=0
temperature=1
raw_request={
    "max_new_tokens":10
}
optional_args={}
stopping_criteria=None

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


torch.Size([6, 19])
10


In [18]:
batch_size = num_generated if batch_size is None else batch_size
assert (num_generated % batch_size)==0
logits=None
sequences=None
for i in range(int(num_generated / batch_size)):
    batch_output = model.generate(
        **encoded_input,
        num_return_sequences=batch_size,
        max_new_tokens=raw_request["max_new_tokens"],

        length_penalty=length_penalty,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        do_sample=True,

        return_dict_in_generate=True,
        output_scores=True,
        output_logits=True,
        **optional_args,
        stopping_criteria=stopping_criteria,
    )


    #generate
    batch_sequences = batch_output.sequences
    batch_logits = torch.stack(list(batch_output.logits), dim=0)

    print("batch!")
    print(batch_sequences.size())
    print(batch_logits.size())

    def safe_append_tensor(tensor_agg, batch_tensor, axis):
        if tensor_agg is None:
            return batch_tensor
        return  torch.cat((tensor_agg,batch_tensor), axis=axis)

    sequences = safe_append_tensor(sequences, batch_sequences, 0)
    logits = safe_append_tensor(logits, batch_logits, 1)




/opt/miniconda3/envs/crfm-helm2/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:698: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


batch!
torch.Size([2, 19])
torch.Size([10, 2, 50257])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


batch!
torch.Size([2, 19])
torch.Size([10, 2, 50257])
batch!
torch.Size([2, 19])
torch.Size([10, 2, 50257])
Final!
torch.Size([6, 19])
torch.Size([10, 6, 50257])


In [13]:
import numpy as np
tensor_batch_logits = np.asarray(batch_logits)
